In [1]:
import json
import random

v2_scenarios = json.load(open("scenario_data/v2_50scenarios.json"))
v3_scenarios = json.load(open("scenario_data/v3_50scenarios.json"))

print(f"Loaded {len(v2_scenarios)} v2 scenarios")
print(f"Loaded {len(v3_scenarios)} v3 scenarios")

Loaded 50 v2 scenarios
Loaded 50 v3 scenarios


The goal of each scenario is to buy a product that:
1. Is of a certain type (e.g. protein bars)
2. Has a certain set of attributes (e.g. gluten-free, low sugar)
3. Is purchased with the correct set of preferences (e.g. when multiple flavors are available, the agent purchases the user's preferred flavor)
4. Is priced under the user's budget

Each scenario is constructed by starting from a single product from the WebShop database and selecting a set of desired attributes, preferences and a budget -- however, there could be other products in the database that also satisfy these criteria, and the agent would be rewarded for purchasing those products as well.

### v2 scenarios

In v2, scenarios were constructed by prompting an LLM to generate an example scenario where a person would might want to purchase the product with the desired attributes. The attributes were not directly mentioned in the scenario text, but can (ideally) be inferred from the scenario or by asking follow-up questions to the users. Users were presented with the scenario text, and also informed about the desired attribute values, the desired product preferences (e.g. size, color) and their budget. 

In [4]:
# Run the following cell to see the v2 scenarios
s = random.choice(v2_scenarios)

out_str = """YOUR CURRENT SHOPPING SCENARIO:
{scenario}
REQUIRED PRODUCT: {product}
REQUIRED ATTRIBUTES: {attributes}
PRODUCT PREFERENCES: 
{preferences}
BUDGET: {budget}"""

print(out_str.format(
    scenario=s["instruction_text"], 
    product=s["product_type"], 
    attributes=', '.join(s["attributes"]), 
    preferences='\n'.join([f"- {k}: {v}" for k, v in s['goal_options_dict'].items()]), 
    budget=f"${s['price_upper']:.0f}")
)



YOUR CURRENT SHOPPING SCENARIO:
You’re planning a Friday pizza night for yourself and two friends, but you all recently started following a low-carb diet and one of your friends can’t eat grains, so you decide to buy frozen cauliflower pizza crusts to make sure everyone can enjoy homemade pizzas together.
REQUIRED PRODUCT: frozen cauliflower pizza crust
REQUIRED ATTRIBUTES: grain free, keto friendly, low calorie, protein serving
PRODUCT PREFERENCES: 
- flavor: original italian
- size: 9.75 ounce (pack of 3)
BUDGET: $40


The problem with the v2 scenarios is that it was a little ambiguous about what the required attributes were, and how they connected to the scenario, so it was possible the agent would buy something with the wrong attribute because it hadn't been instructed by the user correctly.

### v3 Scenarios

In v3, we generated additional reasoning for how all the attributes connected to the scenario. We also did smarter (LLM-based) selection of which attributes to select.

In [3]:
# Run the following cell to see the v3 scenarios

s = random.choice(v3_scenarios)

out_str = """YOUR CURRENT SHOPPING SCENARIO:
{scenario}
REQUIRED PRODUCT: {product}
REQUIRED ATTRIBUTES: {attributes}
REQUIRED PREFERENCES:
{preferences}
BUDGET: {budget}"""

print(out_str.format(
    scenario=s["instruction_text"], 
    product=s["product_type"], 
    attributes="You want the product to have the following attributes:\n" + s['attribute_wise_reasoning'],
    preferences='\n'.join([f"- {k}: {v}" for k, v in s['goal_options_dict'].items()]), 
    budget=f"${s['price_upper']:.0f}")
)

YOUR CURRENT SHOPPING SCENARIO:
You often take tea to work and like to keep a stash in your desk, and you are planning to host a small gathering for your friends who prefer organic beverages; you want to buy green tea bags to have enough for everyone and ensure each bag can be conveniently and cleanly shared.
REQUIRED PRODUCT: green tea bags
REQUIRED ATTRIBUTES: You want the product to have the following attributes:
- Individually wrapped: You want each tea bag to stay fresh and be easy to bring to work or share with friends without making a mess, so individual wrapping is important for cleanliness and convenience.
- Certified organic: Since your friends (and you) prefer organic beverages for health or ethical reasons, you want to choose tea that is certified organic to meet those preferences and provide peace of mind about quality.
REQUIRED PREFERENCES:
- flavor name: green tea bag
- size: 100 count (pack of 1)
BUDGET: $10
